In [24]:

"""
* @ Author: Lucas Glasner (lgvivanco96@gmail.com)
* @ Create Time: 1969-12-31 21:00:00
* @ Modified by: Lucas Glasner,
* @ Modified time: 2024-01-19 15:43:04
* @ Description:
        Routines for the design of flood hydrographs based on synthetic
        hyetographs and unit hydrograph (SUH) methods.
* @ Dependencies:
*/
"""

import pandas as pd
import numpy as np
import scipy.signal as sg
import tqdm

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import xarray as xr
import os

from infiltration import CN_correction
from rain import design_storms, effective_storms
from unithydrographs import SynthUnitHydro
from utils import to_numeric

In [25]:
# ------------------------------- main routines ------------------------------ #
def inputparams(parameters_path, SHyeto_path, tstep=0.5):
    """
    Load input and data an preprocess for the model run

    Args:
        parameters_path (str): Path to textfile with basin parameters. 
        SHyeto_path (str): Path to textfile with synthetic storm data-
        tstep (float, optional): Define simulation timestep. Defaults to 0.1.

    Returns:
        (tuple): formatted parameters,
                 return periods,
                 daily rainfall, 
                 synthetic hyetographs,
                 timestep
    """    
    # Load parameter table
    parameters       = pd.read_csv(parameters_path, index_col=0)
    parameters       = parameters.map(lambda x: to_numeric(x))

    # Get list of hyetograms to use
    parameters.loc['SHyeto'] = parameters.loc['SHyeto'].map(
        lambda x: x.split(';'))

    # Get return periods from the given precipitations
    mask             = parameters.index.map(lambda x: 'pr_' in x) 
    returnperiods_yr = parameters.index[mask]
    returnperiods_yr = returnperiods_yr.map(lambda x: int(x.split('_')[1][1:]))

    # Get precipitations
    rainfall_mm24hr  = parameters[mask]
    rainfall_mm24hr.index = returnperiods_yr

    # Correct curve number by the moisture condition
    parameters.loc['final_curvenumber_1'] = [CN_correction(cn, amc)
                        for cn, amc in zip(parameters.loc['curvenumber_1'],
                                        parameters.loc['moisturecondition_1'])]

    # Set timestep and load synthetic hyetographs
    synth_hyeto    = pd.read_csv(SHyeto_path, index_col=0)
    return parameters, returnperiods_yr, rainfall_mm24hr, synth_hyeto, tstep

def BasinFloodHydrograph(timestep, basin_params, storm_params):
    """
    Main routine to compute a basin flood hydrograph from a set of 
    basin and storm parameters.

    Args:
        timestep (float): Simulation timestep in hours
        basin_params (dict): Basin parameters and geomorpholical properties.
        storm_params (dict): Storm parameters

    Returns:
        _type_: _description_
    """    
    # Compute Unit Hydrograph
    UH    = SynthUnitHydro(method=basin_params['SUnitHydro'],
                           basin_params=basin_params,
                           timestep=timestep)
    uh, uh_params = UH.compute()
    uhname        = uh.name.split('_')[0]
    tstep         = UH.timestep
    
    # Compute design storms
    dhyeto  = design_storms(storm_duration=storm_params['storm_durations'],
                            synth_hyeto=storm_params['synth_hyeto'],
                            tstep=tstep,
                            precips=storm_params['rainfall']).astype(float)
    dhyeto  = effective_storms(dhyeto.pr, CN=basin_params['curvenumber_1'])
    
    # Compute flood hydrograph
    uh = uh.to_xarray().rename({'index':'time'})
    uh = uh.reindex({'time':dhyeto.time})
    uh.attrs = {'standard_name':f'{uhname} unit hydrograph',
                'units':uh.name.split('_')[1],
                'qpeak':uh_params.qpeak.round(2),
                'tpeak':uh_params.tpeak.round(2),
                'tbase':uh_params.tbase.round(2),
                'tstep':uh_params.tstep.round(2)}
    uh.name = 'UnitHydro'
    
    runoff = xr.apply_ufunc(sg.convolve, dhyeto.pr_eff.fillna(0), uh.fillna(0),
                input_core_dims=[['time'],['time']],
                output_core_dims=[['time'],],
                exclude_dims=set([('time'),]),
                keep_attrs=True,
                vectorize=True)
    # runoff = runoff.where(runoff>0)
    runoff.name = 'runoff'
    runoff.coords['time'] = runoff.time*tstep
    runoff.attrs = {'standard_name':'runoff', 'units':'m3*s1'}
    
    model = xr.merge([dhyeto, uh, runoff])
    model.attrs = basin_params
    model.attrs['timestep'] = tstep
    return model

In [252]:
# --------------------------------- Run Model -------------------------------- #
%%time

# Load params
params = load_inputdata('data/parameters.csv', 'data/synthethic_storms.csv')
parameters, returnperiods_yr, rainfall_mm24hr, synth_hyeto, tstep = params

# Iterate over basins and run model
model = {key:None for key in parameters.columns}
for basin_id in tqdm.tqdm(parameters.columns, total=len(parameters.columns)):
    basin_name = parameters[basin_id].loc['name'].replace(' ','')
    try:
        os.mkdir(f'data/Basins/{basin_name}')
        os.mkdir(f'data/Basins/{basin_name}/tmp')
    except:
        pass
    basin_params = parameters[basin_id]
    basin_params['curvenumber_1'] = basin_params['fixed_curvenumber_1']
    basin_params = basin_params.drop('final_curvenumber_1')
    basin_params['zone'] = 'I'
    basin_params = basin_params.to_dict()

    storms = synth_hyeto[basin_params['SHyeto']]
    storm_params = {'storm_durations':[4,6,12,18,24,30,36,42,48,60,72],
                    'synth_hyeto':storms,
                    'rainfall':rainfall_mm24hr[basin_id].dropna()}
    sim = BasinFloodHydrograph(tstep, basin_params, storm_params)
    sim.to_netcdf(f'data/{basin_name}/tmp/run_{basin_name}.nc')
    model[basin_id] = sim


100%|██████████| 7/7 [00:08<00:00,  1.22s/it]

CPU times: user 7.09 s, sys: 135 ms, total: 7.22 s
Wall time: 8.54 s
